In [1]:
MAX_SEQ_LEN = 15


def GlobalParams():
    global MAX_SEQ_LEN

In [2]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
import os

In [3]:
# ① 데이터 읽어오기
train_file = os.path.join('./models/intent', "test_sum.csv")
data = pd.read_csv(train_file, delimiter=',')
data

,intent,query
0,0,"음, 그러면 하나 해주세요."
1,1,네. 자동으로 하면 디씨 되나요?
2,2,어떤 게 있나요?
3,2,네. 내추럴 브라운이랑 두 가지 안 되나요?
4,2,어떤 색깔을 주문해야 되나요?
...,...,...
8222,8,요즘은 유행이 되게 단순하게 많이 나가죠?
8223,4,블라우스나 티셔츠 같은거는 얼마부터 있어요?
8224,4,이런 스카프는 얼마예요?
8225,4,이런 옷은 얼마예요?


In [4]:
data['intent'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [5]:
data[data['intent'] == 7]  # 상품색상문의에 대한 문장

,intent,query
24,7,네 언제 와요?
29,7,네 제가 주말에 갈비탕 두 개를 신청했는데 배송 확인하려고요.
34,7,그럼 내일이나 모레 온다는 거네요?
84,7,언제 오나요?
111,7,이거는 언제 받아 볼 수 있어요?
...,...,...
5674,7,아니 활전복인데 오늘 오전에 와도 불안한데.
5675,7,NaN
5676,7,네. 상품은 오늘이나 내일쯤 오나요?
5677,7,NaN


In [6]:
queries = data.dropna()['query'].tolist()
intents = data.dropna()['intent'].tolist()

In [7]:
data.dropna()['intent'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [8]:
queries[:20]

['음, 그러면 하나 해주세요.',
 '네. 자동으로 하면 디씨 되나요?',
 '어떤 게 있나요?',
 '네. 내추럴 브라운이랑 두 가지 안 되나요?',
 '어떤 색깔을 주문해야 되나요?',
 '아. 다크가 좀 진한 색이죠?',
 '제가 프라이팬으로 착각했지만 이 상품 박스채 그대로라 반품해주세요.',
 '그거 같은 가격에 살 수 있어요?',
 '네, 주문은 맞는데 취소할려고요.',
 '내가 삼십육이니까.',
 '네. 사만 팔천구백원 아닙니까?',
 '바닥에 하는 거 신청하려구요.',
 '이백삼십오말고 이백사십으로 되나요?',
 '예. 타일 박사 신청하려고요.',
 '네 그런데 칠만 사천 백원으로 뜨는 건 뭐예요?',
 '그러면 취소 좀 시켜주세요.',
 '그러니까 배려를 좀 해주세요.',
 '네. 확인이 안 되면 취소해 주세요.',
 '천원 할인이 적용되는 거죠?',
 '육만구천구백원이요?']

In [9]:
intents[:20]

[0, 1, 2, 2, 2, 2, 3, 4, 5, 6, 4, 0, 6, 0, 4, 5, 5, 5, 1, 4]

In [10]:
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

In [11]:
sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

In [12]:
queries[:10]

['음, 그러면 하나 해주세요.',
 '네. 자동으로 하면 디씨 되나요?',
 '어떤 게 있나요?',
 '네. 내추럴 브라운이랑 두 가지 안 되나요?',
 '어떤 색깔을 주문해야 되나요?',
 '아. 다크가 좀 진한 색이죠?',
 '제가 프라이팬으로 착각했지만 이 상품 박스채 그대로라 반품해주세요.',
 '그거 같은 가격에 살 수 있어요?',
 '네, 주문은 맞는데 취소할려고요.',
 '내가 삼십육이니까.']

In [13]:
sequences[:10]

[[320, 708, 2, 685, 25, 244],
 [5, 181, 2, 844, 1019, 13],
 [75, 17, 15],
 [5, 2243, 323, 294, 118, 16, 42, 13],
 [75, 129, 21, 685, 13],
 [7, 1020, 35, 593, 120, 3],
 [63, 1492, 1441, 3, 54, 365, 2244, 536, 178, 10, 685, 25, 244],
 [592, 76, 38, 90, 83, 15],
 [5, 21, 61, 59],
 [212, 119, 157, 3]]

In [14]:
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [15]:
padded_seqs.shape

(8169, 15)

In [16]:
padded_seqs[:10]

array([[ 320,  708,    2,  685,   25,  244,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   5,  181,    2,  844, 1019,   13,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [  75,   17,   15,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   5, 2243,  323,  294,  118,   16,   42,   13,    0,    0,    0,
           0,    0,    0,    0],
       [  75,  129,   21,  685,   13,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   7, 1020,   35,  593,  120,    3,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [  63, 1492, 1441,    3,   54,  365, 2244,  536,  178,   10,  685,
          25,  244,    0,    0],
       [ 592,   76,   38,   90,   83,   15,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   5,   21,   61,   59,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [ 212,  119,  157,    3,    0,

In [17]:
# ③ 학습용, 검증용, 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents)) # 패딩처리된 시퀀스와 의도(intent) 리스트 전체를 데이터셋 객체로
ds = ds.shuffle(len(queries)) # 랜덤 섞기

# 학습셋:검증셋:테스트셋 = 7:2:1
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수

In [18]:
# ④ CNN 모델 정의
# keras 함수형 모델 방식으로 구현
input_layer = Input(shape=(MAX_SEQ_LEN,))  # 입력크기
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(10, name='logits')(dropout_hidden)  # 최종적으로 10개의 의도 클래스를 분류. 결과로 나온 값(logits) 을을 점수(score) 라 부른다
predictions = Dense(10, activation=tf.nn.softmax)(logits)

In [19]:
# ⑤ 모델 생성 
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 15, 128)      374656      ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 15, 128)      0           ['embedding[0][0]']              
                                                                                                  
 conv1d (Conv1D)                (None, 13, 128)      49280       ['dropout[0][0]']                
                                                                                              

In [27]:
# # 모델 학습
# # ★ 시간 걸림 ★

# model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
286/286 [==============================] - 2s 7ms/step - loss: 1.0870 - accuracy: 0.6439 - val_loss: 0.3990 - val_accuracy: 0.9039
Epoch 2/5
286/286 [==============================] - 2s 6ms/step - loss: 0.3918 - accuracy: 0.8949 - val_loss: 0.2578 - val_accuracy: 0.9382
Epoch 3/5
286/286 [==============================] - 2s 6ms/step - loss: 0.2727 - accuracy: 0.9276 - val_loss: 0.1515 - val_accuracy: 0.9608
Epoch 4/5
286/286 [==============================] - 2s 6ms/step - loss: 0.2203 - accuracy: 0.9432 - val_loss: 0.1105 - val_accuracy: 0.9694
Epoch 5/5
286/286 [==============================] - 2s 6ms/step - loss: 0.1748 - accuracy: 0.9556 - val_loss: 0.1213 - val_accuracy: 0.9694


In [45]:
# ⑦ 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

41/41 [==============================] - 0s 2ms/step - loss: 0.0856 - accuracy: 0.9706
Accuracy: 97.058821
loss: 0.085611


In [29]:
# ⑧ 모델 저장
model.save(os.path.join('./models/intent','intent_model.h5'))

In [30]:
from utils.Preprocess import Preprocess    # 의도분류 모델 테스트
from models.intent.IntentModel import IntentModel
import os

p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'user_dic_1.tsv'))

In [31]:
intent = IntentModel(model_name=os.path.join('./models/intent', 'intent_model.h5'), preprocess=p)

In [46]:
query = '제 거 언제와요?'
predict = intent.predict_class(query)
predict_label = intent.labels[predict]
print(query)
print("의도 예측 클래스 : ", predict)
print("의도 예측 레이블 : ", predict_label)

제 거 언제와요?
의도 예측 클래스 :  7
의도 예측 레이블 :  배송일정확인
